# TensorFlow的小知识（随笔）

- <a href='#flags'>1. tf.flags.DEFINE_xxx()</a>
- <a href='#tf_trans'>2. tf.Transform</a>
- <a href='#tf_data'>3. tf.data API</a>

## <a name='flags'>1. tf.flags.DEFINE_xxx()</a>

- tf.flags.DEFINE_xxx() 
- FLAGS = tf.flags.FLAGS 
- FLAGS.flag_values_dict()    
FLAGS._parse_flags()  这种用法，改成了用FLAGS.flag_values_dict()

简单的说：  
用于帮助我们添加命令行的可选参数。    
也就是说利用该函数我们可以实现在命令行中选择需要设定的参数来运行程序， 
可以不用反复修改源代码中的参数，直接在命令行中进行参数的设定。

### 举个例子:

In [2]:
import tensorflow as tf

FLAGS = tf.flags.FLAGS

tf.flags.DEFINE_string('name', 'default', 'name of the model')
tf.flags.DEFINE_integer('num_seqs', 100, 'number of seqs in one batch')
tf.flags.DEFINE_integer('num_steps', 100, 'length of one seq')
tf.flags.DEFINE_integer('lstm_size', 128, 'size of hidden state of lstm')
tf.flags.DEFINE_integer('num_layers', 2, 'number of lstm layers')
tf.flags.DEFINE_boolean('use_embedding', False, 'whether to use embedding')
tf.flags.DEFINE_integer('embedding_size', 128, 'size of embedding')
tf.flags.DEFINE_float('learning_rate', 0.001, 'learning_rate')
tf.flags.DEFINE_float('train_keep_prob', 0.5, 'dropout rate during training')
tf.flags.DEFINE_string('input_file', '', 'utf8 encoded text file')
tf.flags.DEFINE_integer('max_steps', 100000, 'max steps to train')
tf.flags.DEFINE_integer('save_every_n', 1000, 'save the model every n steps')
tf.flags.DEFINE_integer('log_every_n', 10, 'log to the screen every n steps')
tf.flags.DEFINE_integer('max_vocab', 3500, 'max char number')
#全局参数设置，显示在命令行

在命令行中我们为了执行train.py文件，在命令行中输入：

In [ ]:
python train.py \
  --input_file data/shakespeare.txt  \
  --name shakespeare \
  --num_steps 50 \
  --num_seqs 32 \
  --learning_rate 0.01 \
  --max_steps 20000

通过输入不同的文件名、参数，可以快速完成程序的调参和更换训练集的操作，不需要进入源码中更改。

In [1]:
import tensorflow as tf

#取上述代码中一部分进行实验
tf.flags.DEFINE_integer('num_seqs', 100, 'number of seqs in one batch')
tf.flags.DEFINE_integer('num_steps', 100, 'length of one seq')
tf.flags.DEFINE_integer('lstm_size', 128, 'size of hidden state of lstm')

#通过print()确定下面内容的功能
FLAGS = tf.flags.FLAGS  # FLAGS保存命令行参数的数据
FLAGS.flag_values_dict() #将其解析成字典存储到FLAGS.__flags中

{'num_seqs': 100, 'num_steps': 100, 'lstm_size': 128}

In [ ]:
print(FLAGS.flag_values_dict())

# print(FLAGS.num_seqs)

print("\nParameters:")
for attr, value in sorted(FLAGS.flag_values_dict().items()):
    print("{}={}".format(attr.upper(), value))
print("")

### Error信息：UnrecognizedFlagError: Unknown command line flag 'f'
jupyter 下载cifar数据问题：

添加 tf.app.flags.DEFINE_string('f', '', 'kernel') 完美解决！！！

In [1]:
# -*- coding:utf-8 -*-
import os
import csv
import itertools
import functools
import tensorflow as tf
import numpy as np
import pandas as np

# tf.flags.DEFINE_xxx
# FLAGS=tf.flags.FLAGES
# 添加命令行的可选参数
tf.flags.DEFINE_integer(
    'min_word_frequency', 5, 'Minimum frequency of words in the vocabulary'
)

tf.flags.DEFINE_integer(
    'max_sentence_len', 160, "Maximum Sentence Length"
)

tf.flags.DEFINE_string(
    'input_dir', os.path.abspath('../../ubuntu_dataset'),
    "Input directory containing original CSV data files"
)

tf.flags.DEFINE_string(
    'output_dir', os.path.abspath('../../ubuntu_dataset'),
    'Output directory for TFRecord files'
)

# 仅是在jupyter的时候会出现的报错信息。
tf.app.flags.DEFINE_string('f', '', 'kernel')

FLAGS = tf.flags.FLAGS

TRAIN_PATH = os.path.join(FLAGS.input_dir, 'train.csv')
VALIDATION_PATH = os.path.join(FLAGS.input_dir, 'valid.csv')
TEST_PATH = os.path.join(FLAGS.input_dir, 'test.csv')


print(FLAGS.min_word_frequency)

5


----

## <a name='tf_trans'>2. tf.Transform</a>

谷歌发布全新TensorFlow库“tf.Transform” 简化机器学习数据预处理过程。

在实际的机器学习开发中，开发者通常需要对数据集进行大量的耗时费力的预处理过程，以适应各种不同标准的机器学习模型（例如神经网络）。

这些预处理过程根据待解问题的不同和原始数据的组织形式而各不相同，包括不同**格式之间的转换**，**分词**、**词干提取**和**形成词汇**，以及包括**归一化在内的各种数值操作**等等。

实际上，数据的预处理已经成为了机器学习开发中无法回避的一个难题。

针对这一难题，谷歌于 22 日通过开发者博客正式发布了一个基于 TensorFlow 的全新功能组件 —— **tf.Transform**。

它允许用户在大规模数据处理框架中：
- 定义预处理流水线（preprocessing pipelines），
- 同时用户还可以将这些流水线导出，
- 并将其作为 TensorFlow 计算图（TensorFlow graph）的一部分。

用户可以通过组合 Python 函数来定义该流水线，然后在 Apache Beam 框架下通过 tf.Transform 执行。   
（注：Apache Beam 是一个用于大规模的、高效的、分布式的数据处理的开源框架）目前，基于 Apache Beam 框架的流水线可以在 Google Cloud Dataflow 平台上运行，并计划在未来支持更多的平台（可能包括 Apache Apex，Apache Flink 和 Apache Spark 等）。

值得一提的是，通过 tf.Transform 导出的 TensorFlow 计算图还可以在模型预测阶段将这种数据预处理步骤复用（例如，通过 Tensorflow Serving 提供模型时）。

另一方面，开发者最终在产品形态运行机器学习模型时通常还会遇到**“训练服务偏差”（training-serving skew）**，即由于服务中处理的数据与模型训练中使用的数据存在一定的差异，从而造成的预测质量下降。

<img src='./images/training_serve_skew.png' width='90%'/>

面对这一问题，此次新发布的 tf.Transform 组件还能保证预处理过程的零偏差，即**保证服务中的数据预处理和训练中的数据预处理执行完全相同的操作**，特别是当模型训练和服务运行在不同的框架时，例如训练在 TensorFlow，服务在 Apache Beam 的情况。



除了便于**数据的预处理**之外，tf.Transform 还允许用户计算其数据集的**统计概要（summary statistics）**。

对于每个机器学习项目的开发者而言，深刻理解其数据都非常重要，因为任何对底层数据做出的错误假设都可能会产生一些微妙的错误。

通过更简单和高效地计算数据的统计概要，tf.Transform 可以帮助开发者更好地检查他们关于原始数据和预处理数据的假设。

### 安装：
pip install tensorflow-transform

### 官方文档介绍：
https://www.tensorflow.org/tfx/transform/get_started

本指南介绍了它们的基本概念tf.Transform和使用方法。

它会：
- 定义预处理功能，管道的逻辑描述，将原始数据转换为用于训练机器学习模型的数据。
- 通过将预处理功能转换为 Beam管道，显示用于转换数据的Apache Beam实现。
- 显示其他用法示例。

### 1.定义预处理功能

该预处理功能是最重要的概念**tf.Transform**。

预处理功能是数据集转换的逻辑描述。
预处理函数接受并返回张量的字典，其中张量是指Tensor或SparseTensor。

有两种函数用于定义预处理函数：   
1. 任何接受和返回张量的函数。这些将TensorFlow操作添加到图形中，将原始数据转换为转换后的数据。
2. 任何分析仪提供tf.Transform。分析仪也接受并返回张量，但与TensorFlow函数不同，它们不会向图形添加操作。相反，分析器会导致tf.Transform计算TensorFlow之外的全通操作。它们使用整个数据集上的输入张量值来生成作为输出返回的常量张量。例如，tft.min计算数据集上的张量的最小值。tf.Transform提供一组固定的分析仪，但这将在未来的版本中进行扩展。

#### 预处理功能示例
通过组合分析器和常规TensorFlow功能，用户可以创建灵活的管道来转换数据。

以下预处理功能以不同方式转换三个特征中的每一个，并结合了两个特征：

In [2]:
import tensorflow as tf
import tensorflow_transform as tft

ModuleNotFoundError: No module named 'tensorflow_transform'

### 乌龙！安装不上tensorflow_transform
tensorflow-transform requires Python '>=2.7,<3' but the running Python is 3.6.5

-----

## <a name='tf_data'>3. tf.data API</a>
参考网址：https://blog.csdn.net/weixin_38493025/article/details/80672068

通过 tf.data API，您可以根据简单的可重用片段构建复杂的输入管道。

例如：
- 图片模型的管道可能会汇聚分布式文件系统中的文件中的数据、对每个图片应用随机扰动，并将随机选择的图片合并成用于训练的批次。
- 文本模型的管道可能包括从原始文本数据中提取符号、根据对照表将其转换为嵌入标识符，以及将不同长度的序列组合成批次数据。

使用 tf.data API 可以轻松处理大量数据、不同的数据格式以及复杂的转换

### 1. 定义来源

要启动输入管道，您必须定义来源。

例如：
- 要通过内存中的某些张量构建 Dataset，您可以使用 tf.data.Dataset.from_tensors() 或 tf.data.Dataset.from_tensor_slices()
- 如果您的输入数据以推荐的 TFRecord 格式存储在磁盘上，那么您可以构建 tf.data.TFRecordDataset。

一个数据集包含多个元素，每个元素的结构都相同。  
一个元素包含一个或多个 tf.Tensor 对象，这些对象称为组件。 

每个组件都有一个 tf.DType，表示张量中元素的类型；以及一个 tf.TensorShape，表示每个元素（可能部分指定）的静态形状。

In [1]:
import tensorflow as tf

dataset1 = tf.data.Dataset.from_tensor_slices(tf.random_uniform([4,10]))

# 通过 tf.data.Dataset.output_types 和 tf.data.Dataset.output_shapes属性
# 检查数据集元素各个组件的输出类型和形状
print(dataset1.output_types)
print(dataset1.output_shapes)

<dtype: 'float32'>
(10,)


In [2]:
dataset2 = tf.data.Dataset.from_tensor_slices(
    (tf.random_uniform([4]),
    tf.random_uniform([4,100], maxval=100, dtype=tf.int32)))

print(dataset2.output_types)
print(dataset2.output_shapes)


(tf.float32, tf.int32)
(TensorShape([]), TensorShape([Dimension(100)]))


In [3]:
dataset3 = tf.data.Dataset.zip((dataset1, dataset2))

print(dataset3.output_types)
print(dataset3.output_shapes)

(tf.float32, (tf.float32, tf.int32))
(TensorShape([Dimension(10)]), (TensorShape([]), TensorShape([Dimension(100)])))


In [4]:
# 为元素的每个组件命名通常会带来便利性
dataset = tf.data.Dataset.from_tensor_slices(
    {'a': tf.random_uniform([4]),
    'b': tf.random_uniform([4,100], maxval=100, dtype=tf.int32)})

print(dataset.output_types)
print(dataset.output_shapes)

{'a': tf.float32, 'b': tf.int32}
{'a': TensorShape([]), 'b': TensorShape([Dimension(100)])}


### 2. 元素转换

有了 Dataset 对象以后，您就可以通过链接 tf.data.Dataset 对象上的方法调用将其转换为新的 Dataset。

例如：
- 应用单元素转换，例如 Dataset.map()（为每个元素应用一个函数）
- 应用多元素转换（例如 Dataset.batch()）

Dataset 转换支持任何结构的数据集。  
在使用 Dataset.map()、Dataset.flat_map() 和 Dataset.filter() 转换时（这些转换会对每个元素应用一个函数），元素结构决定了函数的参数：

In [5]:
dataset1 = dataset1.map(lambda x: ...)

dataset2 = dataset2.flat_map(lambda x, y: ...)

# Note: Argument destructuring is not available in Python 3.
dataset3 = dataset3.filter(lambda x, (y, z): ...)

### 3. 构建迭代器对象

消耗 Dataset 中值的最常见方法是**构建迭代器对象**。   
通过此对象，可以一次访问数据集中的一个元素（例如通过调用Dataset.make_one_shot_iterator()）。

tf.data.Iterator 提供了两个指令：
- Iterator.initializer，您可以通过此指令（重新）初始化迭代器的状态；
- Iterator.get_next()，此指令返回对应于有符号下一个元素的 tf.Tensor 对象。

#### 3.1 创建单次迭代器

单次迭代器是最简单的迭代器形式，仅支持对数据集进行一次迭代，不需要显式初始化。   

单次迭代器可以处理基于队列的现有输入管道支持的几乎所有情况，但它们不支持参数化。

以 Dataset.range() 为例：

In [25]:
import tensorflow as tf

dataset = tf.data.Dataset.range(100)
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
    for i in range(100):
        value = sess.run(next_element)
        assert i == value

通过assert判断循环是否ok，如果迭代器生成和for循环结果一致，则通过；  
若不通过，则assert报错。

#### 3.2 创建可初始化迭代器

需要先运行显式 iterator.initializer 指令，才能使用可初始化迭代器。

虽然有些不便，但它允许您使用一个或多个 tf.placeholder() 张量（可在初始化迭代器时馈送）参数化数据集的定义。

In [30]:
import tensorflow as tf

max_value = tf.placeholder(tf.int64, shape=[])
dataset = tf.data.Dataset.range(max_value)
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

# Initialize an iterator over a dataset with 10 elements.
with tf.Session() as sess:
    # iterator的初始化，非常重要
    sess.run(iterator.initializer, feed_dict={max_value:10})
    
    for i in range(10):
        value = sess.run(next_element)
        assert i == value

In [32]:
with tf.Session() as sess:
    sess.run(iterator.initializer, feed_dict={max_value: 100})
    for i in range(100):
        value = sess.run(next_element)
        assert i == value

#### 3.3 构建可重新初始化迭代器

可重新初始化迭代器可以通过多个不同的 Dataset 对象进行初始化。

例如：
- 您可能有一个训练输入管道，它会对输入图片进行随机扰动来改善泛化；
- 还有一个验证输入管道，它会评估对未修改数据的预测。

这些管道通常会使用不同的 Dataset 对象，这些对象具有相同的结构（即每个组件具有相同类型和兼容形状）。

In [60]:
import tensorflow as tf

# 定义训练集和验证集，两者有相同的数据结构
training_dataset = tf.data.Dataset.range(100).map(lambda x: x + tf.random_uniform([], -10, 10, tf.int64, seed=1))
validation_dataset = tf.data.Dataset.range(50)

# 一个可重新初始化迭代器通过下面的结构定义。
# 我们可以使用output_types和output_shapes的属性，因为training_dataset和validation_dataset
# 它们两个是兼容的。
iterator = tf.data.Iterator.from_structure(training_dataset.output_types,
                                           training_dataset.output_shapes)
next_element = iterator.get_next()

training_init_op = iterator.make_initializer(training_dataset)
validation_init_op = iterator.make_initializer(validation_dataset)

# Run 20 epochs in which the training dataset is traversed, followed by the validation dataset.
with tf.Session() as sess:
    sess.run(training_init_op)
    for _ in range(10):
        print(sess.run(next_element))

1
-8
6
9
13
8
12
4
14
11


In [61]:
with tf.Session() as sess:
    sess.run(validation_init_op)
    for _ in range(50):
        sess.run(next_element)

#### 3.4 构建可馈送feeding的迭代器

可馈送迭代器可以与 tf.placeholder 一起使用，通过熟悉的 feed_dict 机制来选择每次调用 tf.Session.run 时所使用的 Iterator。 

它提供的功能与可重新初始化迭代器的相同，但在迭代器之间切换时不需要从数据集的开头初始化迭代器。 

例如：  
以上面的同一训练和验证数据集为例，您可以使用 tf.data.Iterator.from_string_handle 定义一个可让您在两个数据集之间切换的可馈送迭代器：

In [1]:
import tensorflow as tf

# Define training and validation datasets with the same structure.
training_dataset = tf.data.Dataset.range(100).map(lambda x: x + tf.random_uniform([], -10, 10, tf.int64, seed=1)).repeat()
validation_dataset = tf.data.Dataset.range(50)

# A feedable iterator is defined by a handle placeholder and its structure. We
# could use the `output_types` and `output_shapes` properties of either
# `training_dataset` or `validation_dataset` here, because they have
# identical structure.
handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle(
    handle, training_dataset.output_types, training_dataset.output_shapes)
next_element = iterator.get_next()

# You can use feedable iterators with a variety of different kinds of iterator
# (such as one-shot and initializable iterators).
training_iterator = training_dataset.make_one_shot_iterator()
validation_iterator = validation_dataset.make_initializable_iterator()


# Loop forever, alternating between training and validation.
with tf.Session() as sess:
    
    # The `Iterator.string_handle()` method returns a tensor that can be evaluated
    # and used to feed the `handle` placeholder.
    training_handle = sess.run(training_iterator.string_handle())
    validation_handle = sess.run(validation_iterator.string_handle())
    
    # Run 200 steps using the training dataset. Note that the training dataset is
    # infinite, and we resume from where we left off in the previous `while` loop iteration.
    for t in range(5):
        for _ in range(200):
            sess.run(next_element, feed_dict={handle: training_handle})

        sess.run(validation_iterator.initializer)
        for _ in range(50):
            sess.run(next_element, feed_dict={handle: validation_handle})
        print('{} times'.format(t))


0 times
1 times
2 times
3 times
4 times


### 4. 消耗迭代器中的值

Iterator.get_next() 方法：   
返回一个或多个 tf.Tensor 对象，这些对象对应于迭代器有符号的下一个元素。   
每次评估这些张量时，它们都会获取底层数据集中下一个元素的值。 
（请注意，与 TensorFlow 中的其他有状态对象一样，调用 Iterator.get_next() 并不会立即使迭代器进入下个状态。相反，您必须使用 TensorFlow 表达式中返回的 tf.Tensor 对象，并将该表达式的结果传递到 tf.Session.run()，以获取下一个元素并使迭代器进入下个状态。）

如果迭代器到达数据集的末尾，则执行 Iterator.get_next() 指令会产生 tf.errors.OutOfRangeError。  
在此之后，迭代器将处于不可用状态；如果需要继续使用，则必须对其重新初始化。

In [8]:
import tensorflow as tf

dataset = tf.data.Dataset.range(5)

iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

# Typically `result` will be the output of a model, or an optimizer's
# training operation.
result = tf.add(next_element, next_element) # 0+0, 1+1, 2+2,...

with tf.Session() as sess:
    sess.run(iterator.initializer)
    print(sess.run(result))
    print(sess.run(result))
    print(sess.run(result))
    print(sess.run(result))
    print(sess.run(result))
    
    try: 
        sess.run(result)
    except tf.errors.OutOfRangeError:
        print('End of dataset')

0
2
4
6
8
End of dataset


### 5. 读取输入数据

#### 5.1 消耗numpy数组

根据 tf.placeholder() 张量定义 Dataset，并在对数据集初始化 Iterator 时馈送 NumPy 数组。

In [ ]:
import tensorflow as tf
import numpy as np

# Load the training data into two NumPy arrays, for example using `np.load()
with np.load('/var/data/training_data.npy') as data:
    features = data['features']
    
# Assume that each row of `features` corresponds to the same row as `labels`.
assert features.shape[0] == labels.shape[0]

features_placeholder = tf.placeholder(features.dtype, features.shape)
labels_placeholder = tf.placeholder(labels.dtype, labels.shape)

dataset = tf.data.Dataset.from_tensor_slices((features_placeholder, labels_placeholder))
# [Other transformations on `dataset`...]
dataset = ...
iterator = dataset.make_initializable_iterator()

sess.run(iterator.initializer, feed_dict={features_placeholder: features,
                                          labels_placeholder: labels})

#### 5.2 消耗 TFRecord 数据

TFRecord 文件格式是一种**面向记录的简单二进制格式**，很多 TensorFlow 应用采用此格式来训练数据。 

通过 tf.data.TFRecordDataset 类，您可以将一个或多个 TFRecord 文件的内容作为输入管道的一部分进行流式传输。 

TFRecordDataset 初始化程序的 filenames 参数可以是字符串、字符串列表，也可以是字符串 tf.Tensor。   
因此，如果您有两组分别用于训练和验证的文件，则可以使用 tf.placeholder(tf.string) 来表示文件名，并使用适当的文件名初始化迭代器：

In [ ]:
import tensorflow as tf

filenames = tf.placeholder(tf.string, shape=None)
dataset = tf.data.TFRecordDataset(filenames)

# Parse the record into tensors.
dataset = dataset.map(lambda x: ...)

# Repeat the input indefinitely.
dataset = dataset.repeat()

dataset = dataset.batch(32)

iterator = dataset.make_initializable_iterator()

# You can feed the initializer with the appropriate filenames for the current
# phase of execution, e.g. training vs. validation.


with tf.Session() as sess:
    # Initialize `iterator` with training data.
    training_filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
    sess.run(iterator.initializer, feed_dict={filenames: training_filenames})
    
    # Initialize `iterator` with validation data.
    validation_filenames = ["/var/data/validation1.tfrecord", ...]
    sess.run(iterator.initializer, feed_dict={filenames: validation_filenames})


#### 5.3 消耗文本数据

很多数据集都是作为一个或多个文本文件分布的。    

tf.data.TextLineDataset 提供了一种从一个或多个文本文件中提取行的简单方法。   
给定一个或多个文件名，TextLineDataset 会为这些文件的每行生成一个字符串值元素。    
像 TFRecordDataset 一样，TextLineDataset 将 filenames 视为 tf.Tensor，因此您可以通过传递 tf.placeholder(tf.string) 来进行参数化。 

In [ ]:
filenames = ["/var/data/file1.txt", "/var/data/file2.txt"]

dataset = tf.data.Dataset.from_tensor_slices(filenames)

# Use `Dataset.flat_map()` to transform each file as a separate nested dataset,
# and then concatenate their contents sequentially into a single "flat" dataset.
# * Skip the first line (header row).
# * Filter out lines beginning with "#" (comments).
# 如果文件以标题行开头或包含评论。
# 可以使用 Dataset.skip() 和 Dataset.filter() 转换来移除这些行
dataset = dataset.flat_map(
    lambda filename: (
        tf.data.TextLineDataset(filename)
        .skip(1)
        .filter(lambda line: tf.not_equal(tf.substr(line, 0, 1), "#"))))